In [1]:
from skimage import io
from skimage.feature import hog
from skimage.feature.texture import local_binary_pattern
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from os import listdir
from os.path import isfile, join
import numpy as np
import cv2
from skimage.color import rgb2gray
from skimage.transform import resize
from sklearn.svm import SVC
from sklearnex import patch_sklearn, config_context
from feature_extraction import (
    get_cold_features,
    get_hinge_features,
    preprocess_image,
    get_contour_pixels,
)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from PIL import ImageEnhance, Image
from numba import jit
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

patch_sklearn()


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
images = []
labels = []

class_1_path = "dataset_resized/flooded"
images = [
    join(class_1_path, f)
    for f in listdir(class_1_path)
    if isfile(join(class_1_path, f))
]
labels = [1 for _ in range(len(images))]

class_2_path = "dataset_resized/non-flooded"
class_2_files = [
    join(class_2_path, f)
    for f in listdir(class_2_path)
    if isfile(join(class_2_path, f))
]
class_2_labels = [0 for _ in range(len(class_2_files))]

images.extend(class_2_files)
labels.extend(class_2_labels)

images = np.array(images)
labels = np.array(labels)

In [3]:
images.shape


(922,)

In [4]:
labels.shape


(922,)

In [5]:
def prepare_image(image_path):
    image = Image.open(image_path)
    image_np = np.array(image)
    image = image.resize((100, 100))
    image_np = rgb2gray(image_np)
    lbp = local_binary_pattern(image_np, 8, 1, method="uniform").flatten()
    hog_features = hog(
        image_np,
        orientations=8,
        pixels_per_cell=(16, 16),
        cells_per_block=(1, 1),
    )
    image, _ = preprocess_image(image)
    contour = get_contour_pixels(image)
    hinge_features = get_hinge_features(contour)
    cold_features = get_cold_features(contour)
    features = np.concatenate([lbp, hog_features, hinge_features, cold_features])
    return features

In [6]:
images = np.array([prepare_image(image_path) for image_path in tqdm(images)])

100%|██████████| 922/922 [01:15<00:00, 12.16it/s]


In [7]:
scaler = StandardScaler()
images = scaler.fit_transform(images)

pca = PCA(n_components=100)
images = pca.fit_transform(images)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

In [12]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
with config_context(target_offload="gpu:0"):
    model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.8162162162162162


In [15]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100)
with config_context(target_offload="gpu:0"):
    model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8432432432432433


In [16]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100)
with config_context(target_offload="gpu:0"):
    model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7945945945945946
